In [2]:
import pandas as pd
import chardet

In [3]:
master_dataframe = pd.read_csv('master_dataframe.csv')

First, design the schema. Identify each piece of information in the schema manually and add it as a variable.

In [4]:
row_index = [4]
complete_record_value = master_dataframe.loc[row_index, 'Complete Records']
complete_record_value_df = complete_record_value.iloc[0]

print(complete_record_value_df)

                                       Complete Record
0                                 CDR FILE INFORMATION
1                   User Entered VIN,1FDXE4FSXCDA06364
2                                              User,SP
3                              Case Number,866715-2017
4    EDR Data Imaging Date,"08/07/2017             ...
..                                                 ...
326                                210.0,-10.10,-16.26
327                                220.0,-10.14,-16.31
328                                 230.0,-9.96,-16.03
329                                 240.0,-9.49,-15.27
330                                 250.0,-9.20,-14.80

[331 rows x 1 columns]


In [ ]:
test_path = r'..\final_project\test_cdr_directory\1FDXE4FSXCDA06364_ACM\1FDXE4FSXCDA06364_ACM.CSV'
with open(test_path, 'rb') as f:
    result = chardet.detect(f.read())
file_encoding = result['encoding']
print(file_encoding)
df = pd.read_csv(test_path, encoding = file_encoding, delimiter = '\r\n', engine = 'python')

In [42]:
import pandas as pd
import chardet

test_path = r'..\final_project\test_cdr_directory\1FDXE4FSXCDA06364_ACM\1FDXE4FSXCDA06364_ACM.CSV'
with open(test_path, 'rb') as f:
    result = chardet.detect(f.read())
file_encoding = result['encoding']
print(file_encoding)

# read in the CSV file and split each row by comma
df = pd.read_csv(test_path, encoding=file_encoding, delimiter='\r\n', engine='python')
#Turning this block into a loop allows exception handling for integers.

ascii


Attempt to extract via CSVs

In [ ]:
import csv
import re

def extract_tables(input_file, output_file):
    # Define the table headers and their corresponding row indices
    headers = {
        "pre_crash_2_sec_2_samples": {"pattern": r"PRE-CRASH DATA -2 SEC \[2 SAMPLES/SEC\] \(FIRST RECORD\)", "row_offset": 1},
        "pre_crash_5_to_0_sec_2_samples": {"pattern": r"PRE-CRASH DATA -5 TO 0 SEC \[2 SAMPLES/SEC\] \(FIRST RECORD\)", "row_offset": 1},
        "pre_crash_5_to_0_sec_10_samples": {"pattern": r"PRE-CRASH DATA -5 TO 0 SEC \[10 SAMPLES/SEC\] \(FIRST RECORD\)", "row_offset": 1},
        "longitudinal_crash_pulse": {"pattern": r"LONGITUDINAL CRASH PULSE \(FIRST RECORD\)", "row_offset": 1},
        "lateral_crash_pulse": {"pattern": r"LATERAL CRASH PULSE \(FIRST RECORD\)", "row_offset": 1}
    }
    
    # Open the input and output files
    with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Find the table headers and their starting row indices
        table_starts = {}
        for i, row in enumerate(reader):
            for header, metadata in headers.items():
                if re.search(metadata["pattern"], row[0], re.IGNORECASE):
                    table_starts[header] = i + metadata["row_offset"]
        
        # Extract the tables and write them to the output file
        for header, start_row in table_starts.items():
            writer.writerow([header])
            for row in reader:
                if len(row) == 0:
                    break
                elif row[0].startswith("Refer to the CDR report for data limitations"):
                    break
                elif len(row) > 1 and row[1] == "":
                    break
                elif reader.line_num < start_row:
                    continue
                elif reader.line_num == start_row:
                    writer.writerow(row)
                else:
                    writer.writerow(row[:-1])
            writer.writerow([])
